# <span style='background :yellow' > MIDAS INTERNSHIP TASK 2 ( Part 3 ) #1</span>

#### By: Yashika Khurana

### Importing libraries

In [63]:
from PIL import Image
import cv2
import os
from PIL import ImageOps
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import keras

### Importing MNIST train data

In [2]:
dftrain=pd.read_csv("mnist_train.csv")

In [3]:
dftrain.head(n=10)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
full=dftrain.values

### Extracting input data and corresponding labels

In [5]:
x=full[:,1:]
y=full[:,0]

In [6]:
x=x.reshape(len(x),28,28,1)

In [7]:
dftest=pd.read_csv("mnist_test.csv")

In [8]:
test=dftest.values

In [9]:
xtest=test[:,1:]
ytest=test[:,0]

In [10]:
xtest=xtest.reshape((len(xtest),28,28,1))

### Building the model

In [11]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
   # keras.layers.BatchNormalization(),
   # keras.layers.Activation(activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),  #1
    keras.layers.Dropout(rate=0.2),  #2
      
    
   
    keras.layers.Flatten(), #3
    keras.layers.Dense( #4
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(11, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [13]:
import keras

In [14]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=3,directory='output',project_name="ykh")

### Searching for best hyper-parameters

In [15]:
tuner_search.search(x,y,epochs=3,validation_split=0.1)

Trial 3 Complete [00h 04m 03s]
val_accuracy: 0.9835000038146973

Best val_accuracy So Far: 0.987666666507721
Total elapsed time: 00h 24m 24s
INFO:tensorflow:Oracle triggered exit


In [16]:
model=tuner_search.get_best_models(num_models=1)[0]

### Training the model on MNIST to generate labels for the training data provided for this task, as provided data does not have labels

In [17]:
model.fit(x, y, epochs=10, validation_split=0.1, initial_epoch=3) 

Epoch 4/10
1688/1688 [==============================] - 345s 204ms/step - loss: 0.0450 - accuracy: 0.9862 - val_loss: 0.0564 - val_accuracy: 0.9867
Epoch 5/10
1688/1688 [==============================] - 389s 230ms/step - loss: 0.0353 - accuracy: 0.9893 - val_loss: 0.0545 - val_accuracy: 0.9880
Epoch 6/10
1688/1688 [==============================] - 2850s 2s/step - loss: 0.0311 - accuracy: 0.9905 - val_loss: 0.0548 - val_accuracy: 0.9878
Epoch 7/10
1688/1688 [==============================] - 288s 171ms/step - loss: 0.0296 - accuracy: 0.9909 - val_loss: 0.0682 - val_accuracy: 0.9858
Epoch 8/10
1688/1688 [==============================] - 294s 174ms/step - loss: 0.0239 - accuracy: 0.9922 - val_loss: 0.0775 - val_accuracy: 0.9858
Epoch 9/10
1688/1688 [==============================] - 284s 169ms/step - loss: 0.0229 - accuracy: 0.9933 - val_loss: 0.0669 - val_accuracy: 0.9868
Epoch 10/10
1688/1688 [==============================] - 347s 205ms/step - loss: 0.0193 - accuracy: 0.9941 - val_l

### Saving the model

In [18]:
model.save("help.h5")

### Generating labels which are the predictions from the model above

In [26]:
data=[]
labels=[]

for i in os.listdir("mnistTask"):
    if str(i)==".DS_Store": # DS_Store file gets created automatically, if-condition helps ignore it
        pass
    else:
    
        for j in os.listdir("mnistTask/"+str(i)):
    
            link="mnistTask/"+str(i)+"/"+str(j)
        
            img=cv2.imread(link)
            img=img[:,:,1]
            data.append(img)
            img=img.reshape((1,28,28,1))
            p=model.predict(img)
            pre=p.argmax()
            labels.append(pre)

In [28]:
len(data)

60000

In [29]:
len(labels)

60000

In [43]:
data=np.array(data)
labels=np.array(labels)

In [44]:
data=data.reshape(len(data),28,28,1)

### Finally training on the MnistTask data provided

In [45]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),

    keras.layers.MaxPooling2D(pool_size=(2, 2)),  #1
    keras.layers.Dropout(rate=0.2),  #2
      
    
    keras.layers.Flatten(), #3
    keras.layers.Dense( #4
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(11, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [46]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [47]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=2,directory='output',project_name="yas")

INFO:tensorflow:Reloading Oracle from existing project output/yas/oracle.json


### Searching for the best model

In [48]:
tuner_search.search(data,labels,epochs=3,validation_split=0.1)

Trial 2 Complete [00h 03m 47s]
val_accuracy: 0.9794999957084656

Best val_accuracy So Far: 0.9794999957084656
Total elapsed time: 00h 07m 03s
INFO:tensorflow:Oracle triggered exit


In [64]:
model=tuner_search.get_best_models(num_models=1)[0]

### Training the model

In [50]:
model.fit(data, labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 75s 44ms/step - loss: 0.0569 - accuracy: 0.9827 - val_loss: 0.0769 - val_accuracy: 0.9795
Epoch 5/10
1688/1688 [==============================] - 72s 43ms/step - loss: 0.0478 - accuracy: 0.9846 - val_loss: 0.0748 - val_accuracy: 0.9812
Epoch 6/10
1688/1688 [==============================] - 74s 44ms/step - loss: 0.0416 - accuracy: 0.9869 - val_loss: 0.1234 - val_accuracy: 0.9792
Epoch 7/10
1688/1688 [==============================] - 83s 49ms/step - loss: 0.0348 - accuracy: 0.9884 - val_loss: 0.0936 - val_accuracy: 0.9788
Epoch 8/10
1688/1688 [==============================] - 85s 50ms/step - loss: 0.0281 - accuracy: 0.9912 - val_loss: 0.1190 - val_accuracy: 0.9760
Epoch 9/10
1688/1688 [==============================] - 82s 49ms/step - loss: 0.0296 - accuracy: 0.9904 - val_loss: 0.1122 - val_accuracy: 0.9802
Epoch 10/10
1688/1688 [==============================] - 80s 47ms/step - loss: 0.0258 - accuracy: 0.9921 - val_loss: 0.1245 

### Importing the test MNIST data for testing

In [51]:
df=pd.read_csv("mnist_test.csv")

In [52]:
full=df.values

### Extracting input data and labels from the dataframe

In [53]:
x=full[:, 1:]
y=full[:,0]

In [54]:
x.shape

(10000, 784)

In [55]:
x=x.reshape(len(x),28,28,1)

### Generating predictions

In [56]:
result=model.predict(x)

In [60]:
ans=[]
for i in range(len(result)):
    r=result[i].argmax()
    ans.append(r)
    
    

In [61]:
ans

[7,
 2,
 1,
 0,
 4,
 1,
 4,
 9,
 5,
 9,
 0,
 6,
 9,
 0,
 1,
 5,
 9,
 7,
 5,
 4,
 9,
 6,
 6,
 5,
 4,
 0,
 7,
 4,
 0,
 1,
 3,
 1,
 3,
 4,
 7,
 2,
 7,
 1,
 2,
 1,
 1,
 7,
 4,
 2,
 3,
 5,
 1,
 2,
 4,
 4,
 6,
 3,
 5,
 5,
 6,
 0,
 4,
 1,
 9,
 5,
 7,
 8,
 5,
 3,
 7,
 4,
 6,
 4,
 3,
 0,
 7,
 0,
 2,
 9,
 1,
 7,
 3,
 2,
 9,
 7,
 7,
 6,
 2,
 7,
 8,
 4,
 7,
 3,
 6,
 1,
 3,
 6,
 9,
 3,
 1,
 4,
 1,
 7,
 6,
 9,
 6,
 0,
 5,
 4,
 9,
 9,
 2,
 1,
 9,
 4,
 8,
 7,
 3,
 9,
 7,
 4,
 4,
 4,
 9,
 2,
 5,
 4,
 7,
 6,
 7,
 9,
 0,
 5,
 8,
 5,
 6,
 6,
 5,
 7,
 8,
 1,
 0,
 1,
 6,
 4,
 6,
 7,
 3,
 1,
 7,
 1,
 8,
 2,
 0,
 2,
 9,
 8,
 5,
 5,
 1,
 5,
 6,
 0,
 3,
 4,
 4,
 6,
 5,
 4,
 6,
 5,
 4,
 5,
 1,
 4,
 4,
 7,
 2,
 3,
 2,
 9,
 1,
 8,
 1,
 8,
 1,
 8,
 5,
 0,
 8,
 9,
 2,
 5,
 0,
 1,
 1,
 1,
 0,
 9,
 0,
 3,
 1,
 6,
 4,
 2,
 3,
 6,
 1,
 1,
 1,
 3,
 9,
 5,
 2,
 9,
 4,
 5,
 9,
 3,
 9,
 0,
 3,
 6,
 5,
 5,
 7,
 2,
 2,
 7,
 1,
 2,
 8,
 4,
 1,
 7,
 3,
 3,
 5,
 8,
 7,
 9,
 2,
 2,
 4,
 1,
 5,
 9,
 8,
 7,
 2,
 3,
 0,
 6,
 4,
 2,


### Finding the test accuracy

In [62]:
score=0
for i in range(len(ans)):
    if(ans[i]==y[i]):
        score=score+1
print((score/len(ans))*100)
        

98.29


### The testing accuracy on MNIST of the randomly initialized network is 98.29